In [ ]:
import requests
import json
import pandas as pd
import logging

logging.basicConfig(level=logging.INFO)

def getData(page, tag):

    base_url = "https://api.stackexchange.com/2.3/questions"
    params = {
        "order": "asc",
        "sort": "creation",
        "tagged": tag,
        "site": "stackoverflow",
        "pagesize": 100,
        "page": page,
        "filter": "withbody",
        "answers": "1",
        "key": "rl_7G8XRSxnP5ZMRRFLme9xurK36"
    }

    try:
        response = requests.get(base_url, params=params)
        response.raise_for_status()
        data = response.json()

        if "items" not in data or not data["items"]:
            logging.warning("No questions found.")
            return None

        data_list = []
        for question in data["items"]:
            answer_id = question.get("accepted_answer_id",None)

            if answer_id:
                answers_url = f"https://api.stackexchange.com/2.3/answers/{answer_id}"
                answers_params = {
                    "site": "stackoverflow",
                    "filter": "withbody",
                    "key": "rl_7G8XRSxnP5ZMRRFLme9xurK36" 
                }
                answers_response = requests.get(answers_url, params=answers_params)
                answers_response.raise_for_status()
                answers_data = answers_response.json()
                answers = answers_data.get('items', [])

                #answers list is not empty and find the first accepted answer
                for answer in answers:
                    if answer.get('is_accepted', False):
                        accepted_answer=answer
                        data_list.append([
                            question.get('title', ''),
                            question.get('body', ''),
                            ", ".join(question.get('tags', [])),
                            accepted_answer.get('body', ''),
                            question.get('score', 0),
                            question.get('creation_date',''),
                            question.get('link','')
                        ])


        df = pd.DataFrame(data_list, columns=["Title", "Description", "Tags", "Accepted Answer", "Question Score", "Creation Time", "Link"])
        
        file_name = "questions_data.csv"

        with open(file_name, 'a', encoding='utf-8', newline='') as f:
              df.to_csv(f, header=None, index=False)

        logging.info(f"Data appended to {file_name}")

    except requests.exceptions.RequestException as req_e:
        logging.error(f"Request failed: {req_e}")
    except json.JSONDecodeError as json_e:
         logging.error(f"Failed to decode JSON response: {json_e}")
    except Exception as e:
        logging.error(f"Unexpected error: {e}")
    return None

In [ ]:
for page in range(1,100):
    result = getData(page,'nlp')